In [1]:
import json, os, pandas as pd, pymongo, spacy, string

/home/maaz-lfd/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-31 19:31:38.014185: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 19:31:49.665446: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-31 19:31:49.666065: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plu

In [3]:
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")

db = mongo_client["cord_literature"]
col_ct = db['clinical_trials']
col_processed_ct = db['processed_clinical_trials']


In [3]:
col_ct

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'cord_literature'), 'clinical_trials')

In [5]:
dataset_path = '/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials'
clinical_path_pmc = os.path.join(dataset_path,'pmc')

In [6]:
CT_dt = pd.read_csv(os.path.join(dataset_path,'CT_metdata.csv'))

In [7]:
len(CT_dt)

8722

In [55]:
# populating paper ID, PUBLISH TIME, JOURNAL, AUTHORS, TITLE, BODY in clinical_trials collection
for index, rows in CT_dt.iterrows():
    pmc_data = json.load(open(os.path.join(clinical_path_pmc,rows['pmcid']+'.xml.json')))
    mongo_dict = {'_id':rows['pmcid'], 'publish_time':rows['publish_time'], 'journal':rows['journal'], 'authors':rows['authors'], 'title':rows['title'], 'abstract':rows['abstract']}
    mongo_dict.update({i['section']: i['text'] for i in pmc_data['body_text']})
    col_ct.insert_one(mongo_dict)

In [10]:
nlp = spacy.load("en_core_web_sm")

In [8]:
def processed_text(text):
    tok_text = [tok.lemma_.lower().strip() for tok in nlp(str(text)) if not tok.like_num and not tok.is_punct]
    tok_text = [tok for tok in tok_text if (tok not in nlp.Defaults.stop_words and tok not in string.punctuation and type(tok) != int and type(tok) != float)]
    return ' '.join(tok_text)

In [14]:
# populating processed text TITLE, BODY in processed_clinical_trials collection
populated_data = [id['_id'] for id in col_processed_ct.find({},{'_id':1})]

for index, rows in CT_dt.iterrows():
    if rows['pmcid'] not in populated_data:
        pmc_data = json.load(open(os.path.join(clinical_path_pmc,rows['pmcid']+'.xml.json')))
        prc_title = processed_text(rows['title']) 
        prc_abstract = processed_text(rows['abstract'])
        prc_body = {i['section']: processed_text(i['text']) for i in pmc_data['body_text']}

        print(rows['title'])
        print(prc_title)
        print('--------------------------------------------------------------')
        print(rows['abstract'])
        print('--------------------------------------------------------------')
        print(prc_abstract)
        print('=================================================================')


        mongo_dict = {'_id':rows['pmcid'], 'title':prc_title, 'abstract':prc_abstract}
        mongo_dict.update(prc_body)
        mongo_dict.update({'all_text':' '.join(mongo_dict[key] for key in mongo_dict if key != '_id') })
        col_processed_ct.insert_one(mongo_dict)

Assessment of ambulance dispatch data for surveillance of influenza-like illness in Melbourne, Australia
assessment ambulance dispatch datum surveillance influenza like illness melbourne australia
--------------------------------------------------------------
OBJECTIVES: Ambulance dispatch data are collated electronically in many jurisdictions and have a wide reach into the community. They may therefore be useful for syndromic surveillance and early recognition of emerging infectious diseases. This study assessed whether ambulance dispatch data are suitable for influenza surveillance. STUDY DESIGN: Comparison of a time series of ambulance dispatch data from Melbourne, Australia for the years 1997–2005 with locum service and general practice (GP) sentinel surveillance data for influenza-like illness (ILI). METHODS: All data were aggregated into 1-week periods, corresponding to the data collection period used in the GP sentinel surveillance system, which was used as the reference system.

In [7]:
a = json.load(open(os.path.join(clinical_path_pmc,'PMC137267.xml.json')))

In [9]:
a.keys()

dict_keys(['paper_id', 'metadata', 'body_text', 'ref_entries', 'back_matter', 'bib_entries'])

In [18]:
for i in a['body_text']:
    print(i['text'])

Anaemia is a common condition in critically ill patients, and RBC transfusions are often used in the treatment and management of this patient population. In fact, one study [1] reported that 25% of all critically ill patients received RBC transfusions. Many laboratory studies [2,3,4,5,6,7,8] have examined the physiological responses (ie compensatory mechanisms) of the body to anaemia, which include the following [9]: increased cardiac output, decreased blood viscosity, capillary changes, increased oxygen extraction, and other tissue adaptations to meet oxygen requirements. Although critically ill patients are affected by a number of factors that predispose them to the adverse consequences of anaemia, persistence of this condition is of particular concern because it may cause the compensatory mechanisms in these patients to become impaired, risking oxygen deprivation in vital organs [9]. However, critically ill patients may also be at increased risk from the adverse effects of RBC trans